<a href="https://colab.research.google.com/github/GenaroHacker/crystal_clear/blob/main/filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#@title Sql
import sqlite3

def CreateTable( DATABASE_NAME , TABLE_NAME , TABLE_STRUCTURE ):
    myConnection=sqlite3.connect(DATABASE_NAME)
    myCursor=myConnection.cursor()
    myCursor.execute('''
        CREATE TABLE ''' + TABLE_NAME + ''' ( ''' + TABLE_STRUCTURE + ''' )
    ''')
    myConnection.commit()
    myConnection.close()

def InsertRecord(DATABASE_NAME, RECORD):
    myConnection=sqlite3.connect(DATABASE_NAME)
    myCursor=myConnection.cursor()
    myCursor.execute(RECORD)
    myConnection.commit()
    myConnection.close()

def InsertSeveralRecords(DATABASE_NAME, MULTIPLE_RECORDS):
    myConnection=sqlite3.connect(DATABASE_NAME)
    myCursor=myConnection.cursor()
    for i in MULTIPLE_RECORDS:
        myCursor.execute(i)
    myConnection.commit()
    myConnection.close()

def ReadRecords(DATABASE_NAME, TABLE_NAME):
    myConnection=sqlite3.connect(DATABASE_NAME)
    myCursor=myConnection.cursor()
    myCursor.execute("SELECT * FROM " + TABLE_NAME)
    records=myCursor.fetchall()
    myConnection.close()
    return records

def ReadLastRecord(DATABASE_NAME, TABLE_NAME):
    myConnection=sqlite3.connect(DATABASE_NAME)
    myCursor=myConnection.cursor()
    myCursor.execute("SELECT * FROM " + TABLE_NAME + " ORDER BY ID DESC LIMIT 1")
    records=myCursor.fetchall()
    myConnection.close()
    return records[0]

def UpdateRecord(DATABASE_NAME, RECORD):
    myConnection=sqlite3.connect(DATABASE_NAME)
    myCursor=myConnection.cursor()
    myCursor.execute(RECORD)
    myConnection.commit()
    myConnection.close()

def RemoveRecord(DATABASE_NAME, RECORD):
    myConnection=sqlite3.connect(DATABASE_NAME)
    myCursor=myConnection.cursor()
    myCursor.execute(RECORD)
    myConnection.commit()
    myConnection.close()

def RunCommand(DATABASE_NAME, COMMAND):
    myConnection=sqlite3.connect(DATABASE_NAME)
    myCursor=myConnection.cursor()
    myCursor.execute(COMMAND)
    myConnection.commit()
    myConnection.close()

def TableNamesToDict(DATABASE_NAME):
    myConnection=sqlite3.connect(DATABASE_NAME)
    myCursor=myConnection.cursor()
    myCursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    records=myCursor.fetchall()
    myConnection.close()
    records_copy = records
    for table in records_copy:
      if table[0] == "TABLE_PARADIGMS":
        records.remove(table)
    records.remove(('sqlite_sequence',))
    records_copy = records
    records = []
    for i in records_copy:
      records.append(i[0])
    d = {}
    counter = 1
    for i in records:
      d[counter] = i
      counter += 1
    return d

def GetEntryAmount(DATABASE_NAME, TABLE_NAME):
    myConnection=sqlite3.connect(DATABASE_NAME)
    myCursor=myConnection.cursor()
    myCursor.execute("SELECT COUNT(*) FROM " + TABLE_NAME)
    records=myCursor.fetchall()
    myConnection.close()
    return records[0][0]



if __name__ == "__main__":
    #Create one Table
    try:
        columns = """
                ID INTEGER PRIMARY KEY AUTOINCREMENT,
                ITEM_NAME VARCHAR(50),
                PRICE INTEGER,
                SECTION VARCHAR(20)"""
        CreateTable("BaseProducts","TableProducts",columns)
        print("Table created successfully!\n")
    except sqlite3.OperationalError:
        print("Table already exists!!!!\n")


    #Insert one record
    InsertRecord("BaseProducts","INSERT INTO TableProducts VALUES (NULL,'BALL',10,'SPORT')")
    print("Record inserted successfully!\n")


    #Insert several records
    InsertSeveralRecords("BaseProducts",[
        "INSERT INTO TableProducts VALUES (NULL,'GOLF STICK',25,'SPORT')",
        "INSERT INTO TableProducts VALUES (NULL,'GLASS',20,'CERAMIC')",
        "INSERT INTO TableProducts VALUES (NULL,'T-SHIRT',5,'CLOTHES')"
    ])
    print("Records inserted successfully!\n")


    #Read all records
    list_of_tuples = ReadRecords("BaseProducts","TableProducts")
    print("The list of tuples with all the records is:")
    print(list_of_tuples)


    #Read last record
    last_record = ReadLastRecord("BaseProducts","TableProducts")
    print("\nThe last record is:")
    print(last_record)


    #Update record
    UpdateRecord("BaseProducts","UPDATE TableProducts SET ITEM_NAME='BALL NAME UPDATED' WHERE ID=3")
    print("\nThe record with ID=3 has been updated successfully!\n")


    #Remove record
    RemoveRecord("BaseProducts","DELETE FROM TableProducts WHERE ID=3")
    print("The record with ID=4 has been removed successfully!\n")

    #Delete table
    RunCommand("BaseProducts","DROP TABLE TableProducts")
    print("\nThe table has been deleted successfully!\n")
    

    #Read all table names
    list_of_table_names = TableNamesToDict("BaseProducts")
    print("The list of table names is:")
    print(list_of_table_names)

    #Get entry amount
    entry_amount = GetEntryAmount("BaseProducts","TableProducts")
    print("\nThe amount of entries in the table is:")
    print(entry_amount)

Table created successfully!

Record inserted successfully!

Records inserted successfully!

The list of tuples with all the records is:
[(1, 'BALL', 10, 'SPORT'), (2, 'GOLF STICK', 25, 'SPORT'), (3, 'GLASS', 20, 'CERAMIC'), (4, 'T-SHIRT', 5, 'CLOTHES')]

The last record is:
(4, 'T-SHIRT', 5, 'CLOTHES')

The record with ID=3 has been updated successfully!

The record with ID=4 has been removed successfully!


The table has been deleted successfully!

The list of table names is:
{}


OperationalError: ignored

In [13]:
#@title Finctions

def delete_paradigm(paradigm_id):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename_todelete = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename_todelete = paradigm[1]
  RunCommand("BaseParadigms","DROP TABLE " + tablename_todelete)

def sort_paradigm(paradigm_id):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename_tosort = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename_tosort = paradigm[1]
  #Read all records
  list_of_tuples = ReadRecords("BaseParadigms",tablename_tosort)
  prdgms_to_sort = []
  for record in list_of_tuples:
    prdgms_to_sort.append(record[1])
  def manual_sort(items_to_sort):
    import functools
    def ask_user_cmp(item1, item2):
        while True:
            print(f"[{item1}](1) or [{item2}](2) ?")
            cmp = input("--> ")
            if cmp == "1":
                return 1
            if cmp == "2":
                return -1
            print("1 or 2, please!")
    ask_user_key = functools.cmp_to_key(ask_user_cmp)
    items_to_sort.sort(key=ask_user_key, reverse=True)
    return items_to_sort
  prdgms_sorted = manual_sort(prdgms_to_sort)
  RunCommand("BaseParadigms","DELETE FROM " + tablename_tosort)
  RunCommand("BaseParadigms","DELETE FROM sqlite_sequence WHERE name='" + tablename_tosort + "'")
  for phrase in prdgms_sorted:
    InsertRecord("BaseParadigms","INSERT INTO " + tablename_tosort + " VALUES (NULL,'" + phrase + "')")

def update_phrase(paradigm_id, phrase_id, new_phrase):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename = paradigm[1]

  #Update record
  UpdateRecord("BaseParadigms","UPDATE " + tablename + " SET PHRASE='" + new_phrase + "' WHERE ID=" + str(phrase_id))





#def delete_phrase(phrase_id)

In [14]:
#delete_paradigm(2)
#sort_paradigm(3)
#update_phrase(3,1,"lol")

In [9]:
#@title Execution


print("Select the paradigm you want to filter or type 0 to select a random paradigm")
#INSERT SHOW ALL PARADIGMS HERE
selected_paradigm = input()
print("")

Select the paradigm you want to filter or type 0 to select a random paradigm
1

